# ハミルトニアンモンテカルロサンプリング

In [1]:
import matplotlib.pyplot as plt
import numpy as np

rng = np.random.default_rng()

## 目的分布: $p(z)$

任意の$z$に対して$\tilde{p}(z) = Z_p p(z)$のみが計算できる: パラメタ$a>0, b>0$が所与のものとして,

$$
\tilde{p}(z | a, b) = b^a z^{a-1} \exp (-bz)
$$

ただし, $z>0$とする.

In [2]:
def tilde_p(z, a, b):
    return (b**a) * (z ** (a-1)) * np.exp(-b * z)

print(tilde_p(-1, 2, 1))

-2.718281828459045


以下では, パラメタ$a, b$について

$$
a = 11 \\
b = 13
$$

とする

In [3]:
a = 11
b = 13

## ハミルトニアンモンテカルロサンプリングの手順

1. ポテンシャルエネルギー$h(z)$の定義
2. 初期値の設定
3. 運動量$p$のサンプリング
4. リープフロッグ法による$z$の計算
5. 確率$r$による採択

### ポテンシャルエネルギーの定義

$$
\begin{align}
h(z)
 &= - \log p(z | a, b) \\
 &= - \log \tilde{p}(z | a, b) + \text{Const.} \\
 &= - \left\lbrace a \log b + (a-1) \log z - bz \right\rbrace + \text{Const.} \\
 &= (1-a) \log z + bz + \text{Const.} \\
\end{align}
$$

なお, 定数項は微分する時消去されるので計算しなくても良い.

ついでに微分も導出しておく.

$$
\begin{align}
\frac{dh(z)}{dz} &= \frac{1-a}{z} + b
\end{align}
$$

In [4]:
def h(z, a, b):
    return (1 - a)*np.log(z) + b*z

def diff_h(z, a, b):
    return (1 - a)/z + b

### 初期値の設定

$$
z^{(0)} = 0.1 \\
\varepsilon = 0.05
$$

とでもしましょうか.

In [5]:
z = 0.1
epsilon = 0.05

### 運動量$p$のサンプリング

$z$は$1$次元なので$p$も$1$次元である.

正規分布からサンプリングするのでお馴染みのPolar法を用いる.

In [6]:
def sample_p(size=1, rng=np.random.default_rng()):
    li = []
    while True:
        u = rng.uniform(-1,1,2)

        s = u[0] ** 2 + u[1] ** 2
        if s > 1: continue

        c = np.sqrt(-2 * np.log(s)/s)
        li.append(c * u[0])
        li.append(c * u[1])

        if len(li) >= size: break

    return np.array(li[:size])

## リープフロッグ法による$z$の計算

$$
\begin{align}
p^{(\tau-1/2)} &= p^{(\tau-1)} - \frac{\varepsilon}{2} \frac{dh(z^{(\tau-1)})}{dz} \\
z^{*} &= z^{(\tau-1)} + \varepsilon p^{(\tau-1/2)} \\
p^{*} &= p^{(\tau-1/2)} - \frac{\varepsilon}{2} \frac{dh(z^{*})}{dz}
\end{align}
$$

In [7]:
def leap_frog(z, p, epsilon, a, b):
    p_mid = p - (epsilon/2) * diff_h(z, a, b)
    z_star = z + epsilon*p_mid
    p_star = p_mid - (epsilon/2) * diff_h(z_star, a, b)

    return (z_star, p_star)

## 確率$r$による採択

$$
r^{(\tau)} = \min \left\{ 1, \exp \left( H(z^{(\tau-1)}, p^{(\tau-1)}) - H(z^{*}, p^{*}) \right) \right\}
$$

なお, $H(z, p)$はハミルトニアンであり, 以下のように計算される.

$$
H(z, p) = h(z) + \frac{1}{2} p^2
$$

In [8]:
def H(z, p, a, b):
    potential = h(z, a, b)
    kinetic = (p**2) / 2
    return potential + kinetic

def rate(z, p, z_star, p_star, a, b):
    k = np.exp(H(z, p, a, b) - H(z_star, p_star, a, b))

    return min([1, k])

In [9]:
burn_in = 1000

for iter in range(burn_in):
    p = sample_p(1, rng)[0]
    z_star, p_star = leap_frog(z, p, epsilon, a, b)
    if rng.uniform() < rate(z, p, z_star, p_star, a, b):
        z = z_star

size = 100000
samples = []

for iter in range(size):
    p = sample_p(1, rng)[0]
    z_star, p_star = leap_frog(z, p, epsilon, a, b)
    if z_star < 0:
        continue
    if rng.uniform() < rate(z, p, z_star, p_star, a, b):
        z = z_star
    samples.append(z)

## 答え合わせ

目的分布$p(z)$は実はガンマ分布$\text{Gam}(z | a, b)$であった:

$$
\text{Gam}(z | a, b) = \frac{b^a z^{a-1} \exp (-bz)}{\Gamma (a)}
$$

これの期待値や分散は以下で与えられる:

$$
\begin{align}
\mathbb{E}[z] &= \frac{a}{b} \\
\text{Var}[z] &= \frac{a}{b^2} \\
\end{align}
$$

サンプリングされた$z$の平均や分散と比較してみる

In [10]:
print("平均, 分散")
print(a/b, a/(b*b))
print(np.mean(samples), np.var(samples))

平均, 分散
0.8461538461538461 0.0650887573964497
0.8465542169607921 0.062929253827916


注意：めちゃくちゃ不安定($\varepsilon$や$a, b$によって結果が良かったり悪かったりする)ので注意。